## Compute source time course (STC) from Raw and Epochs data object
#### Input: *_preprocessed-raw.fif, -epo.fif
####  Output: *.stc

In [10]:
import os
from pathlib import Path
import sys
sys.path.append('/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/')
from eeg_toolkit import preprocess, utils, source_localization
%matplotlib inline

In [11]:
# Settings
times_tup,time_win_path = preprocess.get_time_window(5)

# Read in processed raw data
data_dir = Path("../../Data")
data_path = data_dir / 'Processed Data/'
epo_path = data_dir / time_win_path
stc_path = data_dir / "Source time courses"
EO_resting_save_path = stc_path / "Eyes Open"
EC_resting_save_path = stc_path / "Eyes Closed"
zscored_epochs_save_path = stc_path / "zscored_Epochs" / time_win_path
save_paths = [EC_resting_save_path,EO_resting_save_path,zscored_epochs_save_path] 

# Create save paths
[os.makedirs(path) for path in save_paths if not os.path.exists(path)]

[-2.5,0.0,2.5]


[]

In [12]:
zscored_epochs_save_path

PosixPath('../../Data/Source time courses/zscored_Epochs/5_sec_time_window')

In [13]:
# viable subjects
sub_ids = utils.import_subs(os.path.join(data_path,os.pardir),'sub_ids.txt')
# select only 64ch subs and turn into set
sub_ids = sub_ids[7:]
print(sub_ids)

['018', '020', '021', '022', '023', '024', '027', '029', '030', '031', '032', '033', '034', '035', '036', '037', '038', '039', '040', '041', '042', '043', '044', '045', '046', '048', '049', '050', '051', '052', '053', '054', '055', '056', '057', '058', 'C1.', 'C2.', 'C3.', 'C5.', 'C6.', 'C7.', 'C8.', 'C9.', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C24', 'C25', 'C26', 'C27']


In [14]:
print(f"Chronics: {len([el for el in sub_ids if el.startswith('0')])}")
print(f"Controls: {len([el for el in sub_ids if el.startswith('C')])}")
print(f"Total: {len(sub_ids)}")

Chronics: 36
Controls: 25
Total: 61


In [15]:
# For edge cases of subjects missing eyes open or eyes closed data
no_eyes_open = []
no_eyes_closed = []

In [16]:
# Get stc only from selected labels
roi_names = [# Left
             'rostralanteriorcingulate-lh', # Left Rostral ACC
             'caudalanteriorcingulate-lh', # Left Caudal ACC
             'postcentral-lh', # Left S1,
             'insula-lh', 'superiorfrontal-lh', # Left Insula, Left DL-PFC,
             'medialorbitofrontal-lh', # Left Medial-OFC
             # Right
             'rostralanteriorcingulate-rh', # Right Rostral ACC
             'caudalanteriorcingulate-rh', # Right Caudal ACC
             'postcentral-rh', # , Right S1
             'insula-rh', 'superiorfrontal-rh', # Right Insula, Right DL-PFC
             'medialorbitofrontal-rh'] # Right Medial-OFC

In [17]:
nan_subjects=[]
for sub_id in sub_ids:  
    # Check if eyes open or eyes closed need to be computed
    EC_bool = False if sub_id in no_eyes_open else True
    EO_bool = False if sub_id in no_eyes_closed else True
    
    # Compute source localization for subject and save 
    label_ts_All_Conds, sub_id_if_nan = source_localization.to_source(sub_id,
                                                                      data_path,
                                                                        zscored_epochs_save_path,
                                                                        EC_resting_save_path,
                                                                        EO_resting_save_path,
                                                                        roi_names,
                                                                        times_tup,
                                                                        return_zepochs=True,
                                                                        return_EC_resting= False, #EC_bool,
                                                                        return_EO_resting= False, #EO_bool,
                                                                        average_dipoles=True,
                                                                        )
    # If sub_id(str) output, append to nan_subjects
    if sub_id_if_nan is not None:
        nan_subjects.append(sub_id_if_nan)

Using up to 600 segments
Number of samples used : 48000
[done]
Computing rank from covariance with rank=None
    Using tolerance 6.1e-13 (2.2e-16 eps * 64 dim * 43  max singular value)
    Estimated rank (eeg): 58
    EEG: rank 58 computed from 64 data channels with 1 projector
2 projection items activated
    EEG regularization : 0.1
    Created an SSP operator (subspace dimension = 1)
Computing rank from covariance with rank={'eeg': 58}
    Setting small EEG eigenvalues to zero (without PCA)
Using up to 600 segments
Number of samples used : 48000
[done]
Computing rank from covariance with rank=None
    Using tolerance 1.3e-10 (2.2e-16 eps * 64 dim * 9.4e+03  max singular value)
    Estimated rank (eeg): 55
    EEG: rank 55 computed from 64 data channels with 1 projector
2 projection items activated
    EEG regularization : 0.1
    Created an SSP operator (subspace dimension = 1)
Computing rank from covariance with rank={'eeg': 55}
    Setting small EEG eigenvalues to zero (without PC

/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/eeg_toolkit/source_localization.py:55: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  data_epochs = epochs.get_data()
/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/eeg_toolkit/source_localization.py:57: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  base_data = epochs.get_data(tmin=tmin, tmax=0.0)


Source space          : /home/wanglab/mne_data/MNE-fsaverage-data/fsaverage/bem/fsaverage-ico-5-src.fif
MRI -> head transform : /home/wanglab/mambaforge/envs/mne/lib/python3.11/site-packages/mne/data/fsaverage/fsaverage-trans.fif
Measurement data      : instance of Info
Conductor model   : /home/wanglab/mne_data/MNE-fsaverage-data/fsaverage/bem/fsaverage-5120-5120-5120-bem-sol.fif
Accurate field computations
Do computations in head coordinates
Free source orientations

Reading /home/wanglab/mne_data/MNE-fsaverage-data/fsaverage/bem/fsaverage-ico-5-src.fif...
Read 2 source spaces a total of 20484 active source locations

Coordinate transformation: MRI (surface RAS) -> head
     0.999994  0.003552  0.000202      -1.76 mm
    -0.003558  0.998389  0.056626      31.09 mm
    -0.000001 -0.056626  0.998395      39.60 mm
     0.000000  0.000000  0.000000       1.00

Read  64 EEG channels from info
Head coordinate coil definitions created.
Source spaces are now in head coordinates.

Setting up 

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   8 | elapsed:   12.1s remaining:   20.2s
[Parallel(n_jobs=-1)]: Done   5 out of   8 | elapsed:   12.4s remaining:    7.4s


    Found     0/ 7809 points outside using solid angles
    Total 10242/10242 points inside the surface
Interior check completed in 13036.6 ms
Checking surface interior status for 10242 points...
    Found  2241/10242 points inside  an interior sphere of radius   47.7 mm
    Found     0/10242 points outside an exterior sphere of radius   98.3 mm
    Found     0/ 8001 points outside using surface Qhull


[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:   13.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


    Found     0/ 8001 points outside using solid angles
    Total 10242/10242 points inside the surface
Interior check completed in 1374.7 ms



[Parallel(n_jobs=-1)]: Done   3 out of   8 | elapsed:    1.2s remaining:    2.1s
[Parallel(n_jobs=-1)]: Done   5 out of   8 | elapsed:    1.3s remaining:    0.8s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:    1.4s finished


Setting up for EEG...
Computing EEG at 20484 source locations (free orientations)...


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


### Assess results

In [ ]:
label_ts_All_Conds

In [ ]:
nan_subjects